In [178]:
import inspect
from berny import Berny, geomlib, coords,Geometry,Math
import berny.coords
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
from pyscf.geomopt.berny_solver import to_berny_geom as tbg
ang2bohr=1.8897261246
bohr2ang=.5291772109
np.set_printoptions(precision=3)

In [179]:
berny.coords

<module 'berny.coords' from '/home/administrator/anaconda3/lib/python3.7/site-packages/berny/coords.py'>

In [180]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")

In [181]:
from Morse import mpa
from AP_class import APDFT_perturbator as APP

In [182]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2])),decimals=3)

In [183]:
h2o=gto.M(atom="O 0 0 0; H 0 -1.3 1.1; H 0 1.3 1.5",unit="Bohr",basis="def2-TZVP")

In [184]:
mf=scf.RHF(h2o)
E0=mf.scf()
g0=mf.Gradients().grad()
h0=mf.Hessian().hess()

converged SCF energy = -76.0414421197353
--------------- RHF gradients ---------------
         x                y                z
0 O    -0.0000000000    -0.1009911420    -0.0559574431
1 H     0.0000000000     0.0663066454    -0.0200843097
2 H     0.0000000000     0.0346844966     0.0760417528
----------------------------------------------


In [185]:
h0=h0.swapaxes(1,2)

In [186]:
h0=h0.reshape(9,9)

In [187]:
h20_eq_geom=(optimize(mf).atom_coords())


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.000000  -0.687930   0.582095    0.000000  0.000000  0.000000
   H   0.000000   0.687930   0.793766    0.000000  0.000000  0.000000
converged SCF energy = -76.0414421197384
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000000000    -0.1009912043    -0.0559579130
1 H     0.0000000000     0.0663066114    -0.0200841626
2 H     0.0000000000     0.0346845929     0.0760420756
----------------------------------------------
cycle 1: E = -76.0414421197  dE = -76.0414  norm(grad) = 0.15848

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.046160   0.060851    0.000000  0.046160  0.060851
   H   0.000000  -0.770296   0.565628    0.0000

In [188]:
h20_eq_geom-=h20_eq_geom[0]
minimum=norm(h20_eq_geom[1]),anglecenter(h20_eq_geom)
anglecenter(h20_eq_geom),norm(h20_eq_geom[1]),norm(h20_eq_geom[2])

(1.859, 1.7802514662413118, 1.780307147665229)

In [189]:
h2occords=h2o.atom_coords()

In [190]:
geom2=tbg(h2o)

In [191]:
anglecenter(h2occords),norm(h2occords[1])

(1.583, 1.7029386365926402)

In [192]:
b2=Berny(geom2)
s=b2._state

In [193]:
b2._state.future.q  # bohr and radians

array([1.703, 1.985, 1.583])

In [194]:
q0=s.future.q

In [195]:
s.coords.eval_geom(s.geom)

array([1.703, 1.985, 1.583])

## from dq in internal coords to dq in cartesian coords

In [196]:
B = s.coords.B_matrix(s.geom)
B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
g_ic=np.dot(B_inv.T, (g0).reshape(-1))

In [197]:
g_ic,g0

(array([-0.064,  0.08 , -0.047]), array([[-3.502e-16, -1.010e-01, -5.596e-02],
        [ 1.425e-16,  6.631e-02, -2.008e-02],
        [ 2.078e-16,  3.468e-02,  7.604e-02]]))

In [198]:
np.allclose(B.T@g_ic,g0.flatten())

True

$$B_{\mu i}=\frac{\partial q_\mu}{\partial x_i}$$

In [199]:
############################################
geom0=s.geom.copy()
bms=[]
for i in range(geom0.coords.flatten().shape[0]):
    a=geom0.coords.flatten()
    geom1=geom0.copy()
    a[i]+=.001*bohr2ang
    a=a.reshape(3,3)
    geom1.coords=a
    bms.append((s.coords.B_matrix(geom1)-B)*1000)
bms_arr=np.asarray(bms)
BpG2=np.einsum('ijk,j->ik',bms,g_ic)
h_ic=B_inv.T@(h0-BpG2)@B_inv

In [200]:
norm(h0),norm(h0-B.T@h_ic@B-BpG2) #in realtà la traasformazione non è invertibile, ma ci si può avvicinare intorno al minimo.

(1.7504429966653323, 6.390084963992308e-05)

In [201]:
dq_pr=-np.linalg.inv(h_ic)@g_ic

In [202]:
pred_q=q0+dq_pr
q0+dq_pr,minimum

(array([1.771, 1.668, 1.875]), (1.7802514662413118, 1.859))

In [203]:
coords_new = h2occords+B_inv.dot(dq_pr).reshape(-1, 3)
coords_new-=coords_new[0]

In [204]:
norm(coords_new[1]),anglecenter(coords_new)

(1.809525127906535, 1.877)

In [205]:
dxcc=-(np.linalg.inv(h0)@g0.flatten()).reshape(3,3)
dxcc-=dxcc[0]
norm((h2occords+dxcc)[1]),anglecenter(h2occords+dxcc)

(2.379773933641184, 1.555)

In [206]:
dxcc=-(np.linalg.solve(h0,g0.flatten())).reshape(3,3)
dxcc-=dxcc[0]
norm((h2occords+dxcc)[1]),anglecenter(h2occords+dxcc)

(2.3797738972048603, 1.555)

In [207]:
app=APP(mf,sites=[1])
app.build_all()

In [208]:
depE=app.APDFT3(np.asarray([-1]))-app.mf.e_tot

In [209]:
mpa(g_ic[0],h_ic[0,0],s.future.q[0],depE)[0]-q0[0],dq_pr[0],-g_ic[0]/h_ic[0,0]

(0.08510200589216166, 0.06840963081731409, 0.07799798163649954)

In [210]:
ddq_mb=np.asarray([mpa(g_ic[0],h_ic[0,0],q0[0],depE)[0]+g_ic[0]/h_ic[0,0]-s.future.q[0],\
                mpa(g_ic[1],h_ic[1,1],q0[1],depE)[0]+g_ic[1]/h_ic[1,1]-q0[1],0])
ddq_mb
q0,q0+dq_pr,q0+dq_pr+ddq_mb,minimum

(array([1.703, 1.985, 1.583]),
 array([1.771, 1.668, 1.875]),
 array([1.778, 1.719, 1.875]),
 (1.7802514662413118, 1.859))

In [211]:
#with tabled BDE
mpa(g_ic[0],h_ic[0,0],s.future.q[0],119/627)[0]+g_ic[0]/h_ic[0,0]+dq_pr[0]

1.7847215795422742

In [212]:
tbde=119/627
ddq_mb=np.asarray([mpa(g_ic[0],h_ic[0,0],q0[0],tbde)[0]+g_ic[0]/h_ic[0,0]-s.future.q[0],\
                mpa(g_ic[1],h_ic[1,1],q0[1],tbde)[0]+g_ic[1]/h_ic[1,1]-q0[1],0])
ddq_mb
q0,q0+dq_pr,q0+dq_pr+ddq_mb,minimum

(array([1.703, 1.985, 1.583]),
 array([1.771, 1.668, 1.875]),
 array([1.785, 1.761, 1.875]),
 (1.7802514662413118, 1.859))

In [213]:
def build_h_ic(s,g_ic,h0):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(3,3)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic

In [214]:
tbbde={"OH":119/627,"HO":119/627}
def mpa_pb(coords,atoml,g,h):
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0],g.shape[0])
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    g_ic=np.dot(B_inv.T, (g).reshape(-1)) 
    h_ic=build_h_ic(s,g_ic,h)
    dq_NR=-np.linalg.inv(h_ic)@g_ic
    ddq_mb=np.zeros_like(dq_NR)
    bnr=0
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bondatoms=atoml[s.coords._coords[i].i]+atoml[s.coords._coords[i].j]
            ddq_mb[i]+=mpa(g_ic[i],h_ic[i,i],q0[i],tbbde[bondatoms])[0]+g_ic[i]/h_ic[i,i]-q0[i]
            bnr+=1
    #ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],g_ic[bnr:]+h_ic[bnr:,:bnr]@(dq_NR)[:bnr])
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    return q0,q0+dq_NR,q0+dq_NR+ddq_mb,ddq_mb
    #ddq_mb=np.asarray([mpa(g_ic[0],h_ic[0,0],q0[0],tbde)[0]+g_ic[0]/h_ic[0,0]

In [215]:
mpa_pb(h2o.atom_coords(),[h2o.atom_symbol(x) for x in range(h2o.natm)],g0,h0)

(array([1.703, 1.985, 1.583]),
 array([1.771, 1.668, 1.875]),
 array([1.785, 1.761, 1.852]),
 array([ 0.013,  0.093, -0.023]))

In [216]:
minimum


(1.7802514662413118, 1.859)